*A seperate window will pop up with vizualisations when you run the simulation

# Assumptions 

- Only 1 surgeon
- Only 2 exam rooms 
- Surgeon can only help 1 patient at a time (can only be in 1 exam room at a time)
- Patients always arrive on time 
- No patients need any procedures 
- Time to walk to exam room is small and constant 
- Patients always enter empty exam rooms / wait in the shortest line for exam rooms
- Exam room time follows an exponential distribution 

# Simulation 

In [7]:
# -------------------------
#  PACKAGES
# -------------------------

import itertools
import functools
from collections import defaultdict

import random
import numpy as np
import pandas as pd
import math
import time

import simpy

import json

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

import tkinter as tk
from PIL import ImageTk

# -------------------------
#  CONFIGURATION
# -------------------------

# Schedule is in minutes
# Enter schedule from earliest to latest 
PATIENT_SCHEDULE = {"time":[1,27,43,71,99, 131, 160, 199, 211], "surgeon_id":[1,1,1,1,1,1,1,1,1]} 

TIME_TO_WALK_TO_SURGEON_MEAN = 0.1
TIME_TO_WALK_TO_SURGEON_STD = 0
TIME_TO_WALK_TO_EXAM_MEAN = 0.1
TIME_TO_WALK_TO_EXAM_STD = 0

SURGEONS = 1
SURGEONS_PER_LINE = 1
SURGEON_MEAN = 0
SURGEON_STD = 0

EXAM_LINES = 2
EXAM_ROOM_PER_LINE = 1
EXAM_MEAN = 12
EXAM_STD = 2

random.seed(424)

# -------------------------
#  ANALYTICAL GLOBALS
# -------------------------

arrivals = defaultdict(lambda: 0)
utilization = defaultdict(lambda: 0)
surgeon_waits = defaultdict(lambda: [])
exam_waits = defaultdict(lambda: [])
exam_time = defaultdict(lambda: [])
event_log = []
exam_ends = {"patient_id":[], "exam_end":[]}

def register_arrivals(time, num):
    arrivals[int(time)] += num

def register_utilization(time, service_time):
    utilization [int(time)] += service_time
    
def register_surgeon_wait(time, wait):
    surgeon_waits[int(time)].append(wait)

def register_exam_wait(time, wait):
    exam_waits[int(time)].append(wait)    
    
def register_exam_time(time, service_time):
    exam_time[int(time)].append(service_time)

def avg(raw_waits):
    waits = [ w for i in raw_waits.values() for w in i ]
    return round(np.mean(waits), 1) if len(waits) > 0 else 0

def register_patient_arrival(time, patient_id, people_created):
    register_arrivals(time, len(people_created))
    print(f"Patient #{patient_id} arrived at {round(time,2)}")
    event_log.append({
        "event": "PATIENT_ARRIVAL",
        "time": round(time, 2),
        "patientId": patient_id,
        "peopleCreated": people_created
    })

def register_patient_going_to_surgeon(people, walk_begin, walk_end, surgeon, queue_begin, queue_end, surg_begin, surg_end):
    wait = queue_end - queue_begin
    service_time = surg_end - surg_begin
    register_surgeon_wait(queue_end, wait)
    event_log.append({
        "event": "WALK_TO_SURGEON",
        "people": people,
        "surgeon": surgeon,
        "time": round(walk_begin, 2),
        "duration": round(walk_end - walk_begin, 2)
    })
    event_log.append({
        "event": "WAIT_IN_SURGEON_LINE",
        "people": people,
        "surgeon": surgeon,
        "time": round(queue_begin, 2),
        "duration": round(queue_end - queue_begin, 2)
    })
    event_log.append({
        "event": "SEE SURGEON",
        "people": people,
        "surgeon": surgeon,
        "time": round(surg_begin, 2),
        "duration": round(surg_end - surg_begin, 2)
    })

def register_patient_moving_to_exam(person, walk_begin, walk_end, exam_line, util, queue_begin, queue_end, exam_begin, exam_end, patient_id, patient_surgeon_id, service_time):
    wait = queue_end - queue_begin
    register_exam_time(queue_end, service_time)
    register_exam_wait(queue_end, wait)
    util_data= util.data[-1]
    register_utilization(util_data[0], service_time)
    print(f"Patient #{patient_id} waited {round(wait,2)} minutes for exam room {exam_line}, needed {round(service_time,2)} minutes to complete")
    print(f"Patient #{patient_id} left at {round(exam_end,2)}")
    exam_ends["patient_id"].append(patient_surgeon_id)
    exam_ends["exam_end"].append(exam_end)
    event_log.append({
        "event": "WALK_TO_EXAM",
        "person": patient_id,
        "examLine": exam_line,
        "time": round(walk_begin, 2),
        "end": round(walk_end,2)
    })
    event_log.append({
        "event": "WAIT_IN_EXAM_LINE",
        "person": patient_id,
        "examLine": exam_line,
        "time": round(queue_begin, 2),
        "end": round(queue_end, 2)
    })
    event_log.append({
        "event": "EXAM_ROOM",
        "person": patient_id,
        "examLine": exam_line,
        "time": round(exam_begin, 2),
        "end": round(exam_end, 2)
    })

# -------------------------
#  VIZUALISATION 
# -------------------------

main = tk.Tk()
main.title("Patient Simulation")
main.config(bg="#fff")
top_frame = tk.Frame(main)
top_frame.pack(side=tk.TOP, expand = False)
canvas = tk.Canvas(main, width = 1300, height = 350, bg = "white")
canvas.pack(side=tk.TOP, expand = False)

f = plt.Figure(figsize=(2, 2), dpi=72)
a3 = f.add_subplot(121)
a3.plot()
a1 = f.add_subplot(222)
a1.plot()
a2 = f.add_subplot(224)
a2.plot()
data_plot = FigureCanvasTkAgg(f, master=main)
data_plot.get_tk_widget().config(height = 400)
data_plot.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

class QueueGraphics:
    text_height = 30
    icon_top_margin = -8
    
    def __init__(self, icon_file, icon_width, queue_name, num_lines, canvas, x_top, y_top):
        self.icon_file = icon_file
        self.icon_width = icon_width
        self.queue_name = queue_name
        self.num_lines = num_lines
        self.canvas = canvas
        self.x_top = x_top
        self.y_top = y_top

        self.image = tk.PhotoImage(file = self.icon_file, master = main)
        self.icons = defaultdict(lambda: [])
        for i in range(num_lines):
            canvas.create_text(x_top, y_top + (i * self.text_height), anchor = tk.NW, text = f"{queue_name} Queue  #{i + 1}")
        self.canvas.update()

    def add_to_line(self, surgeon_number):
        count = len(self.icons[surgeon_number])
        x = self.x_top + 120 + (count * self.icon_width)
        y = self.y_top + ((surgeon_number - 1) * self.text_height) + self.icon_top_margin
        self.icons[surgeon_number].append(
                self.canvas.create_image(x, y, anchor = tk.NW, image = self.image)
        )
        self.canvas.update()

    def remove_from_line(self, surgeon_number):
        if len(self.icons[surgeon_number]) == 0: return
        to_del = self.icons[surgeon_number].pop()
        self.canvas.delete(to_del)
        self.canvas.update()

def Surgeons(canvas, x_top, y_top):
    return QueueGraphics("person-resized.gif", 25, "Surgeon", SURGEONS, canvas, x_top, y_top)

def Exams(canvas, x_top, y_top):
    return QueueGraphics("person-resized.gif", 18, "Exam Room", EXAM_LINES, canvas, x_top, y_top)

class PatientLog:
    TEXT_HEIGHT = 24
    
    def __init__(self, canvas, x_top, y_top):
        self.canvas = canvas
        self.x_top = x_top
        self.y_top = y_top
        self.patient_count = 0
    
    def next_patient(self, minutes, time, patient_id):
        x = self.x_top
        y = self.y_top + (self.patient_count * self.TEXT_HEIGHT)
        self.canvas.create_text(x, y, anchor = tk.NW, text = f" Patient {patient_id} in {round(minutes-time, 1)} minutes")
        self.canvas.update()
    
    def patient_arrived(self, people):
        x = self.x_top + 140
        y = self.y_top + (self.patient_count * self.TEXT_HEIGHT)
        self.canvas.create_text(x, y, anchor = tk.NW, text = f"Arrived", fill = "green")
        self.patient_count = self.patient_count + 1
        self.canvas.update()

class ClockAndData:
    def __init__(self, canvas, x1, y1, x2, y2, time):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.canvas = canvas
        self.train = canvas.create_rectangle(self.x1, self.y1, self.x2, self.y2, fill="#fff")
        self.time = canvas.create_text(self.x1 + 10, self.y1 + 10, text = "Time = "+str(round(time, 1))+"m", anchor = tk.NW)
        self.exam_wait = canvas.create_text(self.x1 + 10, self.y1 + 40, text = "Avg. Wait for Exam = "+str(avg(exam_waits)), anchor = tk.NW)
        self.exam_time = canvas.create_text(self.x1 + 10, self.y1 + 70, text = "Avg. Time in Exam = "+str(avg(exam_time)), anchor = tk.NW)
        self.canvas.update()

    def tick(self, time):
        self.canvas.delete(self.time)
        self.canvas.delete(self.exam_time)
        self.canvas.delete(self.exam_wait)

        self.time = canvas.create_text(self.x1 + 10, self.y1 + 10, text = "Time = "+str(round(time, 1))+"m", anchor = tk.NW)
        self.exam_wait = canvas.create_text(self.x1 + 10, self.y1 + 30, text = "Avg. Wait for Exam = "+str(avg(exam_waits))+"m", anchor = tk.NW)
        self.exam_time = canvas.create_text(self.x1 + 10, self.y1 + 50, text = "Avg. Time in Exam = "+str(avg(exam_time))+"m", anchor = tk.NW)
        
        a1.cla()
        a1.set_xlabel("Time")
        a1.set_ylabel("Avg. Wait for Exam (minutes)")
        a1.step([ t for (t, waits) in exam_waits.items() ], [ np.mean(waits) for (t, waits) in exam_waits.items() ])
        
        a2.cla()
        a2.set_xlabel("Time")
        a2.set_ylabel("Avg. Time in Exam Room (minutes)")
        a2.step([ t for (t, service_time) in exam_time.items() ], [ np.mean(service_time) for (t, service_time) in exam_time.items() ])
        
        a3.cla()
        a3.set_xlabel("Time")
        a3.set_ylabel("Utilization Rate")
        a3.step([t for (t, service_time) in utilization.items()],  [np.cumsum([list(utilization.items()).index((t, service_time))]) / t for (t, service_time) in utilization.items()])
        data_plot.draw()
        self.canvas.update()

patient_log = PatientLog(canvas, 5, 20)
surgeon_lines = Surgeons(canvas, 340, 20)
exams = Exams(canvas, 770, 20)
clock = ClockAndData(canvas, 1100, 260, 1290, 340, 0)

# -------------------------
#  SIMULATION
# -------------------------

class MonitoredResource(simpy.Resource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []
    def request(self, *args, **kwargs):
        self.data.append((self._env.now, self.count))
        return super().request(*args, **kwargs)
    def release(self, *args, **kwargs):
        self.data.append((self._env.now, self.count))
        return super().release(*args, **kwargs)
            
def pick_shortest(lines):
    """
        Given a list of SimPy resources, determine the one with the shortest queue.
        Returns a tuple where the 0th element is the shortest line (a SimPy resource),
        and the 1st element is the line # (1-indexed)
        Note that the line order is shuffled so that the first queue is not disproportionally selected
    """
    shuffled = list(zip(range(len(lines)), lines)) # tuples of (i, line)
    random.shuffle(shuffled)
    shortest = shuffled[0][0]
    for i, line in shuffled:
        if len(line.queue) < len(lines[shortest].queue):
            shortest = i
            break
    return (lines[shortest], shortest + 1)

def create_clock(env):
    """
        This generator is meant to be used as a SimPy event to update the clock
        and the data in the UI
    """
    
    while True:
        yield env.timeout(0.1)
        clock.tick(env.now)

def patient_arrival(env, surgeons, exam_lines):
    next_patient_id = 1
    next_person_id = 0
    while len(PATIENT_SCHEDULE["time"]) > 0:
        next_patient = (PATIENT_SCHEDULE["time"].pop(0),PATIENT_SCHEDULE["surgeon_id"].pop(0))
        patient_surgeon_id = next_patient[1]
        on_board = 1
        
        patient_log.next_patient(next_patient[0], env.now, next_patient_id)
        yield env.timeout(next_patient[0]-env.now)
        patient_log.patient_arrived(on_board)
        
        people_ids = list(range(next_person_id, next_person_id + on_board))
        register_patient_arrival(env.now, next_patient_id, people_ids)
        next_person_id += on_board
        next_patient_id += 1
        people_processed = [1]
        surgeon_queue = surgeon_id["surgeon_queue"][surgeon_id["surgeon_id"].index(patient_surgeon_id)]
        env.process(surgeon_control(env, people_processed, surgeons, exam_lines, next_patient_id-1, patient_surgeon_id, surgeon_queue))


                    
def see_surgeons(env, people_processed,surgeons, exam_lines, next_patient_id, patient_surgeon_id):
    walk_begin = env.now
    yield env.timeout(random.gauss(TIME_TO_WALK_TO_SURGEON_MEAN, TIME_TO_WALK_TO_SURGEON_STD))
    walk_end = env.now

    queue_begin = env.now
    surgeon = pick_shortest(surgeons)
    with surgeon1.request() as req:
        # Wait in line
        surgeon_lines.add_to_line(surgeon[1])
        yield req
        surgeon_lines.remove_from_line(surgeon[1])
        queue_end =env.now

        surg_begin = env.now
        yield env.timeout(random.gauss(SURGEON_MEAN, SURGEON_STD))
        surg_end = env.now

        register_patient_going_to_surgeon(people_processed, walk_begin, walk_end, surgeon[1], queue_begin, queue_end, surg_begin, surg_end)
        
        env.process(exam_room(env, people_processed, exam_lines, TIME_TO_WALK_TO_EXAM_MEAN, TIME_TO_WALK_TO_EXAM_STD, next_patient_id, patient_surgeon_id))

surgeon_id = {"surgeon_id":[1], "surgeon_queue":[see_surgeons]}

def surgeon_control(env, people_processed, surgeons, exam_lines, next_patient_id, patient_surgeon_id, surgeon_queue):
    if next_patient_id not in exam_ends["patient_id"]:
        yield env.process(surgeon_queue(env, people_processed,surgeons, exam_lines, next_patient_id, patient_surgeon_id)) 
    yield env.timeout(1)
            
def exam_room(env, people_processed, exam_lines, walk_duration, walk_std, next_patient_id, patient_surgeon_id):
    walk_begin = env.now
    yield env.timeout(random.gauss(walk_duration, walk_std))
    walk_end = env.now

    # We assume that the visitor will always pick the shortest line
    queue_begin = env.now    
    exam_line = pick_shortest(exam_lines)
    with exam_line[0].request() as req:
        # Wait in line
        for _ in people_processed: exams.add_to_line(exam_line[1])
        yield req
        for _ in people_processed: exams.remove_from_line(exam_line[1])
        queue_end = env.now
         
        for person in people_processed:
            exam_begin = env.now
            yield env.timeout(np.random.exponential(EXAM_MEAN))
            exam_end = env.now
            service_time = exam_end - exam_begin
            register_patient_moving_to_exam(person, walk_begin, walk_end, exam_line[1], exam_line[0], queue_begin, queue_end, exam_begin, exam_end, next_patient_id, patient_surgeon_id, service_time)


env = simpy.Environment()


surgeon1 = simpy.Resource(env, capacity = 1)
surgeons = [ simpy.Resource(env, capacity = SURGEONS_PER_LINE) for _ in range(SURGEONS) ]
exam_lines = [ MonitoredResource(env, capacity = EXAM_ROOM_PER_LINE) for _ in range(EXAM_LINES) ]

env.process(patient_arrival(env, surgeons, exam_lines))
env.process(create_clock(env))

env.run(until = 240)

main.mainloop()

Patient #1 arrived at 1
Patient #1 waited 0.0 minutes for exam room 1, needed 7.0 minutes to complete
Patient #1 left at 8.2
Patient #2 arrived at 27
Patient #2 waited 0.0 minutes for exam room 1, needed 2.99 minutes to complete
Patient #2 left at 30.19
Patient #3 arrived at 43
Patient #3 waited 0.0 minutes for exam room 1, needed 15.8 minutes to complete
Patient #3 left at 59.0
Patient #4 arrived at 71
Patient #4 waited 0.0 minutes for exam room 2, needed 1.65 minutes to complete
Patient #4 left at 72.85
Patient #5 arrived at 99
Patient #6 arrived at 131
Patient #6 waited 0.0 minutes for exam room 2, needed 3.77 minutes to complete
Patient #6 left at 134.97
Patient #5 waited 0.0 minutes for exam room 1, needed 37.27 minutes to complete
Patient #5 left at 136.47
Patient #7 arrived at 160
Patient #7 waited 0.0 minutes for exam room 1, needed 3.59 minutes to complete
Patient #7 left at 163.79
Patient #8 arrived at 199
Patient #9 arrived at 211
Patient #8 waited 0.0 minutes for exam room 